In [1]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten

import cv2

import pandas as pd
import numpy as np
import matplotlib.image as mpimg

from utils import DataFrame

Using TensorFlow backend.
/Users/mithi/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Model Parameters 
BATCH_SIZE = 128
NUMBER_OF_EPOCHS = 5
VALIDATION_SIZE = 0.15
OPTIMIZER_TYPE = "adam"
LOSS_TYPE = "mse"
INPUT_H, INPUT_W = 75, 200
INPUT_SHAPE = (INPUT_H, INPUT_W, 3)
# Crop parameters
YSTART = 110
YSTOP = 230

print("=========")
left_red_frame = DataFrame("./road-images/red.csv")
print("=========")
right_green_frame = DataFrame("./road-images/green.csv")


csv filepath:  ./road-images/red.csv
number of images:  8949
sample data: 
                                                NAME  STEER
0  ./road-images/red-flat/eagle_2018_09_14_20_29_...    0.0
1  ./road-images/red-flat/eagle_2018_09_14_20_29_...    0.0
2  ./road-images/red-flat/eagle_2018_09_14_20_29_...    0.0
3  ./road-images/red-flat/eagle_2018_09_14_20_29_...    0.0
4  ./road-images/red-flat/eagle_2018_09_14_20_29_...    0.0


csv filepath:  ./road-images/green.csv
number of images:  8824
sample data: 
                                                NAME  STEER
0  ./road-images/green-flat/eagle_2018_09_14_20_2...    0.0
1  ./road-images/green-flat/eagle_2018_09_14_20_2...    0.0
2  ./road-images/green-flat/eagle_2018_09_14_20_2...    0.0
3  ./road-images/green-flat/eagle_2018_09_14_20_2...    0.0
4  ./road-images/green-flat/eagle_2018_09_14_20_2...    0.0



In [3]:
def random_flip(image, steer):
    if np.random.random() > 0.5:
        return np.fliplr(image), -steer
    else:
        return image, steer

In [4]:
def get_processed_data(x, data):
    
    i = data.index[x]
    steer =  data['STEER'][i]
    img = mpimg.imread(data['NAME'][i])
    img = img[YSTART:YSTOP, :, :]
    img = cv2.resize(img, (INPUT_W, INPUT_H), cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    
    return random_flip(img, steer)

In [5]:
def generate_samples(data, batch_size):

    while True:

        SIZE = len(data)
        data.sample(frac=1)

        for start in range(0, SIZE, batch_size):
            images, steers = [], []

            for i in range(start, start + batch_size):
                if i < SIZE:
                    image, steer = get_processed_data(i, data)
                    steers.append(steer)
                    images.append(image)

            yield (np.array(images), np.array(steers))         

In [6]:
def build_modified_nvidia_model():
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE))
    model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    model.compile(optimizer = OPTIMIZER_TYPE, loss = LOSS_TYPE)
    return model

In [7]:
def train_model(model, data, modelh5_name, modeljson_name):
    
    print('summary of model:')
    model.summary()
    
    print('Training model...')

    TRAINING_DATA, VALIDATION_DATA = train_test_split(data, test_size=VALIDATION_SIZE)
    TOTAL_TRAIN_DATA = len(TRAINING_DATA)
    TOTAL_VALID_DATA = len(VALIDATION_DATA)

    training_generator = generate_samples(TRAINING_DATA, batch_size=BATCH_SIZE)
    validation_generator = generate_samples(VALIDATION_DATA, batch_size=BATCH_SIZE)

    model.fit_generator(training_generator,
        samples_per_epoch=TOTAL_TRAIN_DATA,
        validation_data=validation_generator,
        nb_val_samples=TOTAL_VALID_DATA,
        nb_epoch=NUMBER_OF_EPOCHS,
        verbose=1)

    print('...Model trained.')
    
    print('Saving model...')
    
    model.save(modelh5_name)

    with open(modeljson_name, "w") as json_file:
        json_file.write(model.to_json())

    print("...Model Saved.")
    
    return model

In [8]:
left_red_model = build_modified_nvidia_model()

In [9]:
left_red_model = train_model(left_red_model, left_red_frame.data, "./models/left_red.h5", "./models/left_red.json")

summary of model:
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 75, 200, 3)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 36, 98, 24)    1824        lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 16, 47, 36)    21636       convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 6, 22, 48)     43248       convolution2d_2[0][0]            
_________________________________________________________________________

In [11]:
right_green_model = build_modified_nvidia_model()
right_green_model = train_model(right_green_model, right_green_frame.data, "./models/right_green.h5", "./models/right_green.json")

summary of model:
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_3 (Lambda)                (None, 75, 200, 3)    0           lambda_input_3[0][0]             
____________________________________________________________________________________________________
convolution2d_11 (Convolution2D) (None, 36, 98, 24)    1824        lambda_3[0][0]                   
____________________________________________________________________________________________________
convolution2d_12 (Convolution2D) (None, 16, 47, 36)    21636       convolution2d_11[0][0]           
____________________________________________________________________________________________________
convolution2d_13 (Convolution2D) (None, 6, 22, 48)     43248       convolution2d_12[0][0]           
_________________________________________________________________________